In [1]:
import csv
import chess
rows = []
with open('lichess_db_puzzle.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        rows.append(row)

#row[0] = PuzzleID
#row[1] = FEN
#row[2] = Moves
#row[3] = Rating
#row[4] = RatingDeviation
#row[5] = Popularity
#row[6] = NbPlayes
#row[7] = Themes
#row[8] = URL
#row[9] = OpeningTags
#row[10] = GoodRowThemes
#row[11] = CountGoodRowThemes
header = rows[0]
rows.remove(rows[0])
#print(f"{header[1]:<70}{header[2]:<40}{header[3]:<15}{header[5]:<15}{header[6]:<15}{header[7]:<30}")
#for row in rows:
#    print(f"{row[1]:<70}{row[2]:<40}{row[3]:<15}{row[5]:<15}{row[6]:<15}{row[7]:<30}")




In [2]:
'''
The puzzles are filtered for our requirements.
This was done without a pipeline to better understand the what was being filtered
'''

final = []
for row in rows[1:]:
    if not(int(row[5]) < 70 or int(row[6]) < 1000): #Adds every puzzle from the initial database if the popularity is at least 70 and was played at least 1000 times.
        final.append(row)
print("number of puzzles: ",len(final))
    

number of puzzles:  1088967


In [ ]:
'''
Here we filter out the first unwanted themes, that we didn't need at all.
'''

final2 = []
for row in final:
    if not("superGM" in row[7] or "master" in row[7] or "arabianMate" in row[7] or "bodenMate" in row[7] or "doubleBishopMate" in row[7] or "hookMate" in row[7] or "oneMove" in row[7] or "quietMove" in row[7] or "underPromotion" in row[7] or "zugzwang" in row[7]):
        final2.append(row)

print("number of puzzles: ",len(final2))

number of puzzles:  913536


In [4]:
'''
In the puzzle database their is no theme for which pieces are moved.
We add these themes in this section, by importing chess (in cell 1) and checking which piece is on the solution square.
The solution squares are given in the database.
'''

piece_names = {
    'P': 'Pawn',
    'N': 'Knight',
    'B': 'Bishop',
    'R': 'Rook',
    'Q': 'Queen',
    'K': 'King'
}
themes = set()
for row in final2:
    moves = row[2].split() #Get the separate moves
    board = chess.Board(row[1]) #Build the board using the FEN - a way to represent the whole position of a chess game
    used_pieces = set() #We need this set, as in some puzzles the player has to move the same piece twice and we don't want to have a duplicate theme
    for i in range(len(moves)):
        move = chess.Move.from_uci(moves[i])
        if i % 2 == 1:  # Check if the index is odd (every second move). We check because the puzzle starts with the move of an opponent, so the players moves are 2, 4, ...
            piece = board.piece_at(move.from_square).symbol().upper()
            if piece not in used_pieces:  # Check if the piece is new
                used_pieces.add(piece)  # Add new piece to the set
                row[7] += " " + piece_names.get(piece, "Unknown") #This adds the piece to the themes. "Unknown" is never used, as we checked for it in usedPieces, but there for stylistic choices
        board.push(move) #this "does" the move, changing the board position
    rowThemes = row[7].split()
    goodRowThemes = []
    #Here we decided to remove further themes because the total number of themes would have resulted in a very large training set.
    #Unlike the filtering before, we don't remove the row completely from our puzzle list, we just make a separate list entry with the good themes
    badThemes = ['short','attackingF2F7','knightEndgame', 'pawnEndgame','mate','bishopEndgame','dovetailMate','queenEndgame','backRankMate','anastasiaMate','rookEndgame','veryLong','smotheredMate','long','queenRookEndgame']
    for theme in rowThemes:
        if (theme not in badThemes):
            themes.add(theme)
            goodRowThemes.append(theme)
    row.append(goodRowThemes)
    row.append(len(goodRowThemes))

print(themes)
print(len(themes))
  

{'mateIn3', 'advantage', 'advancedPawn', 'endgame', 'skewer', 'doubleCheck', 'middlegame', 'promotion', 'clearance', 'queensideAttack', 'xRayAttack', 'Bishop', 'intermezzo', 'mateIn5', 'exposedKing', 'kingsideAttack', 'opening', 'discoveredAttack', 'defensiveMove', 'sacrifice', 'fork', 'Rook', 'enPassant', 'castling', 'capturingDefender', 'pin', 'attraction', 'trappedPiece', 'Pawn', 'deflection', 'Knight', 'hangingPiece', 'interference', 'crushing', 'Queen', 'King', 'mateIn4', 'mateIn2'}
38


In [5]:
#Get New CSV File with all important Puzzles and the important Themes filtered out
header.append('GoodRowThemes')
header.append('CountGoodRowThemes')
final2.insert(0, header)
with open('ImportantPuzzles.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the data
    writer.writerows(final2)

In [ ]:
'''
This part is the initial puzzle generation.
We filter through the list to get possible puzzles within a rating range of +-80 given the participants previous rating.
Then we find puzzles, such that every theme appears at least 'TimesToOccur' times.

We save only the link to the puzzles in a XLSX file for the participant and save the other information about the puzzles in a CSV file that is not send to the participants.

This part is very inefficient, this was changed for the final puzzle design.
It is inefficient because we randomly select puzzles, meaning we will check some puzzles twice or more often.

The findLessThanX part is also very inefficient, because we go through all our selected puzzles and make a new directory every time we call the function.
This can be made more efficient by looking at the themes we are adding and removing them from the list by hand, that way we can make it in constant time.
'''

import random
import xlsxwriter
import numpy as np

def getPossiblePuzzles(rating: int):
    possiblePuzzles = []
    for row in final2[1:]:
        if (int(row[3]) > rating -80 and int(row[3]) < rating +80): #This is the rating range
            possiblePuzzles.append(row)
    
    return possiblePuzzles

def findLessThanX(selectedPuzzles2, timesToOccur):
    theme_count = {} #directory key-value pairs

    # Count the occurrences of each theme
    for puzzle in selectedPuzzles2:
        for theme in puzzle[10]:
            if theme in theme_count:
                theme_count[theme] += 1 #add 1 to count if already has a counter
            else:
                theme_count[theme] = 1 #add a counter and set to 1
    lessThanXCount = [] #filter through themes that occur less often than 'TimesToOccur'
    for theme in themes:
        if(theme not in theme_count or theme_count[theme] < timesToOccur):
            lessThanXCount.append(theme)
    return lessThanXCount, theme_count

def ensure_themes_appear_X_times(initialPuzzles2, timesToOccur):

    lessThanX, theme_count = findLessThanX(initialPuzzles2, timesToOccur) #Here we get the puzzles that appear less than 3 times, initially all puzzles

    # Add puzzles to the list, focusing on underrepresented themes
    while (len(lessThanX) > 0):
        randomID = random.randint(0,len(possiblePuzzles)-1) 
        puzzle = possiblePuzzles[randomID] #Get a random puzzle from the list.
        if (puzzle not in initialPuzzles2): #Check if the puzzle is already being used
            for puzzleTheme in puzzle[10]: 
                if puzzleTheme in lessThanX: #Check if the puzzle theme occurs less than 'TimesToOccur' times
                    for puzzleTheme2 in puzzle[10]:
                        if theme in theme_count:
                            theme_count[theme] += 1 #add 1 to count if already has a counter
                        else:
                            theme_count[theme] = 1 #add a counter and set to 1
                    initialPuzzles2.append(puzzle)
                    lessThanX, theme_count = findLessThanX(initialPuzzles2, timesToOccur) #update needed themes
                    break


    return initialPuzzles2

def write_to_excel(initialIDs, filename):
    workbook = xlsxwriter.Workbook(filename)
    worksheet = workbook.add_worksheet()

    # Write the data
    for row_num, row_data in enumerate(initialIDs):
        for col_num, cell_data in enumerate(row_data):
            worksheet.write(row_num, col_num, cell_data)

    workbook.close()

for i in [1300, 1950, 1750, 1800]:  #The initial puzzle rating of the participants

    possiblePuzzles = getPossiblePuzzles(i)
    initialPuzzles = []
    timesToOccur = 4
    countXInitialPuzzles = ensure_themes_appear_X_times(initialPuzzles, timesToOccur)

    randomShuffling = np.random.choice(range(0, len(countXInitialPuzzles)), size=len(countXInitialPuzzles), replace=False)

    countXInitialPuzzlesShuffled = [countXInitialPuzzles[i] for i in randomShuffling]

    initialIDs = []
    for puzzle in countXInitialPuzzlesShuffled:
        initialIDs.append (["=HYPERLINK(\"https://lichess.org/training/" + puzzle[0] + "\")", ""])

    #Get New CSV File with all initial Puzzles
    initialHeader = ["PuzzleLink","Richtig(1) oder Falsch(0)"]
    initialIDs.insert(0, initialHeader)

    countXInitialPuzzlesShuffled.insert(0, header)
    playerPuzzles = 'InitialPuzzles' + str(i) + '.csv'
    with open(playerPuzzles, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the data
        writer.writerows(countXInitialPuzzlesShuffled)

    playerIDs = 'InitialIDs' + str(i) + '.xlsx'
    write_to_excel(initialIDs, playerIDs)


Anzahl Count 15 Puzzles: 341 Rating:  2000
